### All standard imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import seaborn as sns
from sklearn.decomposition import PCA
from scipy.stats import mode
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import silhouette_samples, silhouette_score
plt.rc('font', size = 11)
sns.set_style('darkgrid')
sns.set_palette(sns.color_palette('Paired'))
sns.palplot(sns.color_palette('Paired'))
sns.set()

### Preprocessing the dataset

In [ ]:
#import pandas as pd
dataset_read = pd.read_excel('pilot_experiment_TPM_WTonly.xlsx') #reading the dataset using pandas read_excel function
unprocessed_dataset = dataset_read.iloc[:, 0:].T #Transpose of the raw dataset
y = dataset_read.columns.values.tolist() #getting the label/target labels as list from the dataset 

y_categorized = [] 
sample_list = []
#assigning categorical label to 0-9 (ascending order) to y_categorized as label for each sample
for i in y:
    if i[:-2] == 'Ox_Leaf1_T1':
        y_categorized.append(0)
        sample_list.append('L1T1')
    elif i[:-2] == 'Ox_Leaf1_T2':
        y_categorized.append(1)
        sample_list.append('L1T2')
    elif i[:-2] == 'Ox_Leaf1_T3':
        y_categorized.append(2)
        sample_list.append('L1T3')
    elif i[:-2] == 'Ox_Leaf1_T4':
        y_categorized.append(3)
        sample_list.append('L1T4')
    elif i[:-2] == 'Ox_Leaf3_T2':
        y_categorized.append(4)
        sample_list.append('L3T2')
    elif i[:-2] == 'Ox_Leaf3_T3':
        y_categorized.append(5)
        sample_list.append('L3T3')
    elif i[:-2] == 'Ox_Leaf3_T4':
        y_categorized.append(6)
        sample_list.append('L3T4')
    elif i[:-2] == 'Ox_Leaf5_T3':
        y_categorized.append(7)
        sample_list.append('L5T3')
    elif i[:-2] == 'Ox_Leaf5_T4':
        y_categorized.append(8)
        sample_list.append('L5T4')
    elif i[:-2] == 'Ox_Leaf7_T4':
        y_categorized.append(9)
        sample_list.append('L7T4')

unprocessed_dataset['y'] = y_categorized #putting the label information with the dataset, as the dataset does not contain the label
dataset = unprocessed_dataset
X = dataset.iloc[:, :-1].values #getting the dataset without label, where each row represents sample, each column represents featues or independent variables
y = dataset.iloc[:, -1].values #label column from the data set

#for the plotting with the labels
#import numpy as np
sample_arr = np.array(sample_list)
set_sample = set(sample_list)
set_sample = sorted(list(set_sample))

### Different feature scaling methods:
#### Standard Scalar, MinMax Scalar, Quantile Transformer, Normalizer

#### StandardScaler:
removes the mean and scales the data to unit variance. However, the 
outliers have an influence when computing the empirical mean and standard deviation 
which shrink the range of the feature values as shown in the left figure below. Note in 
particular that because the outliers on each feature have different magnitudes, the 
spread of the transformed data on each feature is very different.
StandardScaler therefore cannot guarantee balanced feature scales in the presence of outliers.

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc_X_train = sc.fit_transform(X)
print('For standard Scalar, the max value for each row is same:', np.amax(sc_X_train, axis = 1))

#### Minmax Scalar:
rescales the data set such that all feature values are in the range [0, 1]
#As StandardScaler, MinMaxScaler is very sensitive to the presence of outliers.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
mms_X_train = mms.fit_transform(X)

#### QuantileTransformer
has an additional output_distribution parameter allowing to match a 
Gaussian distribution instead of a uniform distribution. Note that this non-parametetric 
transformer introduces saturation artifacts for extreme values.

In [ ]:
from sklearn.preprocessing import QuantileTransformer
qt = QuantileTransformer(output_distribution = 'uniform')
qt_X_train = qt.fit_transform(X)

#### The Normalizer
rescales the vector for each sample to have unit norm, independently of 
the distribution of the samples. It can be seen on both figures below where all samples 
are mapped onto the unit circle. In our example the two selected features have only 
positive values; therefore the transformed data only lie in the positive quadrant. This 
would not be the case if some original features had a mix of positive and negative values.

In [ ]:
from sklearn.preprocessing import Normalizer
nr = Normalizer()
nr_X_train = nr.fit_transform(X)

### Visualization of the scaled data by reducing the dimensionality with PCA

In [ ]:
#import numpy as np
#import matplotlib.pyplot as plt
#from sklearn.decomposition import PCA
pca_visualization = PCA(n_components = 2)
#Using the PCA on my scaled features
X_transformed = pca_visualization.fit_transform(sc_X_train) 
#transforming my features to 2 dimension(feature extraction from my orginal feature set)

In [ ]:
plt.figure(figsize = (14,10))
plt.scatter(X_transformed[:,0], X_transformed[:,1], c = y, s = 50, cmap = 'tab10', alpha = 0.7)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.colorbar()
plt.title('Each color represent each sample set')
#plt.savefig('C:/Users/Tamal/Documents/Thesis Files/Images/Kmeans/Normalized_Quantile_PCA-transformed.png', dpi = 200)
plt.show()

### Applying Kmeans algorithm to the scaled dataset

In [ ]:
#from sklearn.cluster import KMeans
#import matplotlib.pyplot as plt
Kmeans = KMeans(n_clusters = 10, init = 'k-means++', max_iter = 300, n_init = 100) #creating the Kmeans object
Kmeans.fit(sc_X_train)
y_kmeans = Kmeans.fit_predict(sc_X_train) #fitting the learning model to the data and predicting the clusters for the samples
plt.rc('font', size = 10) #setting the front size in the plot
fig, ax = plt.subplots(figsize = (14,10)) #figure size
plt.scatter(X_transformed[y_kmeans == 0, 0], X_transformed[y_kmeans == 0, 1], s = 300, c = 'red', label = 'Cluster 1', alpha = 0.5)
plt.scatter(X_transformed[y_kmeans == 1, 0], X_transformed[y_kmeans == 1, 1], s = 300, c = 'blue', label = 'Cluster 2', alpha = 0.5)
plt.scatter(X_transformed[y_kmeans == 2, 0], X_transformed[y_kmeans == 2, 1], s = 300, c = 'green', label = 'Cluster 3', alpha = 0.5)
plt.scatter(X_transformed[y_kmeans == 3, 0], X_transformed[y_kmeans == 3, 1], s = 300, c = 'cyan', label = 'Cluster 4', alpha = 0.5)
plt.scatter(X_transformed[y_kmeans == 4, 0], X_transformed[y_kmeans == 4, 1], s = 300, c = 'silver', label = 'Cluster 5', alpha = 0.5)
plt.scatter(X_transformed[y_kmeans == 5, 0], X_transformed[y_kmeans == 5, 1], s = 300, c = 'peru', label = 'Cluster 6',alpha = 0.5)
plt.scatter(X_transformed[y_kmeans == 6, 0], X_transformed[y_kmeans == 6, 1], s = 300, c = 'lawngreen', label = 'Cluster 7', alpha = 0.5)
plt.scatter(X_transformed[y_kmeans == 7, 0], X_transformed[y_kmeans == 7, 1], s = 300, c = 'lightgreen', label = 'Cluster 8', alpha = 0.5)
plt.scatter(X_transformed[y_kmeans == 8, 0], X_transformed[y_kmeans == 8, 1], s = 300, c = 'pink', label = 'Cluster 9', alpha = 0.5)
plt.scatter(X_transformed[y_kmeans == 9, 0], X_transformed[y_kmeans == 9, 1], s = 300, c = 'purple', label = 'Cluster 10', alpha = 0.5)
for i, txt in enumerate(sample_list):
    ax.annotate(txt, (X_transformed[:,0][i], X_transformed[:,1][i]))
plt.legend()
#fig.savefig('C:/Users/Tamal/Documents/Thesis Files/Images/Kmeans/Kmeans_Quantile_NormalizedData.png', dpi = 200)
plt.show()

#### By applying feature scaling Standard Scalar, MinMax Scalar, Quantile Transformer all the three samples from L5_T3, L7_T4 were correctly clustered which is unlikely without scaled feature. Still L3_T2 samples were clustered in 2 different clusters.And as for raw input with or without PCA dimensionality reduction and scaled features L3_T4 and L5_T4 were always clustred together.

In [ ]:
# Using the dendrogram to find the optimal number of clusters
import scipy.cluster.hierarchy as sch

plt.figure(figsize = (14,10))
dendrogram = sch.dendrogram(sch.linkage(sc_X_train, method = 'ward'), labels = sample_list)
plt.title('Dendrogram')
plt.xlabel('Customers')
plt.ylabel('Euclidean distances')
plt.show()

In [ ]:
# Fitting Hierarchical Clustering to the dataset
from sklearn.cluster import AgglomerativeClustering

hc = AgglomerativeClustering(n_clusters = 10, affinity = 'euclidean', linkage = 'ward')
y_hc = hc.fit_predict(sc_X_train)

plt.rc('font', size = 10) #setting the front size in the plot
fig, ax = plt.subplots(figsize = (14,10)) #figure size
plt.scatter(X_transformed[y_hc == 0, 0], X_transformed[y_hc == 0, 1], s = 300, c = 'red', label = 'Cluster 1', alpha = 0.5)
plt.scatter(X_transformed[y_hc == 1, 0], X_transformed[y_hc == 1, 1], s = 300, c = 'blue', label = 'Cluster 2', alpha = 0.5)
plt.scatter(X_transformed[y_hc == 2, 0], X_transformed[y_hc == 2, 1], s = 300, c = 'green', label = 'Cluster 3', alpha = 0.5)
plt.scatter(X_transformed[y_hc == 3, 0], X_transformed[y_hc == 3, 1], s = 300, c = 'cyan', label = 'Cluster 4', alpha = 0.5)
plt.scatter(X_transformed[y_hc == 4, 0], X_transformed[y_hc == 4, 1], s = 300, c = 'silver', label = 'Cluster 5', alpha = 0.5)
plt.scatter(X_transformed[y_hc == 5, 0], X_transformed[y_hc == 5, 1], s = 300, c = 'peru', label = 'Cluster 6',alpha = 0.5)
plt.scatter(X_transformed[y_hc == 6, 0], X_transformed[y_hc == 6, 1], s = 300, c = 'lawngreen', label = 'Cluster 7', alpha = 0.5)
plt.scatter(X_transformed[y_hc == 7, 0], X_transformed[y_hc == 7, 1], s = 300, c = 'lightgreen', label = 'Cluster 8', alpha = 0.5)
plt.scatter(X_transformed[y_hc == 8, 0], X_transformed[y_hc == 8, 1], s = 300, c = 'pink', label = 'Cluster 9', alpha = 0.5)
plt.scatter(X_transformed[y_hc == 9, 0], X_transformed[y_hc == 9, 1], s = 300, c = 'purple', label = 'Cluster 10', alpha = 0.5)
for i, txt in enumerate(sample_list):
    ax.annotate(txt, (X_transformed[:,0][i], X_transformed[:,1][i]))
plt.legend()
#plt.savefig('C:/Users/Tamal/Documents/Thesis Files/Images/Kmeans/KMeans_Rawdata.png', dpi = 200)
plt.show()

### Evaluation of the learned Kmeans model on the scaled dataset

In [ ]:
labels = np.zeros_like(y_kmeans)
for i in range(10):
    mask = (y_kmeans == i)
    labels[mask] = mode(y[mask])[0]

acc_score = accuracy_score(y, labels)
print('The accuracy score for optimized K-means algorithm {}.'.format(acc_score))

mat = confusion_matrix(y, labels)
sns.set(rc={'figure.figsize':(12,8)})
sns.heatmap(mat.T, square = False, annot = True, fmt = 'd', cbar = False, xticklabels = set_sample, yticklabels = set_sample)
plt.xlabel('True label')
plt.ylabel('Predicted label')
#plt.savefig('C:/Users/Tamal/Documents/Thesis Files/Images/Kmeans/ConfusionMatrix_Kmeans_Quantile_NormalizedData.png', dpi = 200)
plt.show()

#### As the samples from L3_T2 were clustered in 2 clusters without clustering any other samples from different clusters, the confusion matrix count them as correctly classified. As so 90% accuracy was acheived.

### Applying PCA on the scaled data to reduce dimenspionality that contains 99.99% variance of the data

In [ ]:
#import numpy as np
#import matplotlib.pyplot as plt
#from sklearn.decomposition import PCA
pca = PCA(.9999)
#transforming my features to 2 dimension(feature extraction from my orginal feature set)
X_trans = pca.fit_transform(sc_X_train)
num_components = pca.n_components_
print('{} components retain 99.99% variance of the data and shape {}.'.format(num_components, X_trans.shape))

In [ ]:
#import numpy as np
#import matplotlib.pyplot as plt
#from sklearn.decomposition import PCA
plt.figure(figsize = (14,10))
plt.scatter(X_trans[:,0], X_trans[:,1], c = y, s = 50, cmap = 'tab10', alpha = 0.7)
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.colorbar()
plt.title('Each color represent each sample set')
#fig.savefig('C:/Users/Tamal/Documents/Thesis Files/Images/Kmeans/Kmeans_Quantile_NormalizedData_PCA_reduction.png', dpi = 200)
plt.show()

### Applying Kmeans algorithm to the scaled reduced dimension dataset

In [ ]:
#from sklearn.cluster import KMeans
#import matplotlib.pyplot as plt
Kmeans = KMeans(n_clusters = 10, init = 'k-means++', max_iter = 300, n_init = 100) #creating the Kmeans object
Kmeans.fit(X_trans) #Reduced scaled feature set
y_kmeans = Kmeans.fit_predict(X_trans) #fitting the learning model to the data and predicting the clusters for the samples
plt.rc('font', size = 10) #setting the front size in the plot
fig, ax = plt.subplots(figsize = (14,10)) #figure size
plt.scatter(X_trans[y_kmeans == 0, 0], X_trans[y_kmeans == 0, 1], s = 300, c = 'red', label = 'Cluster 1', alpha = 0.5)
plt.scatter(X_trans[y_kmeans == 1, 0], X_trans[y_kmeans == 1, 1], s = 300, c = 'blue', label = 'Cluster 2', alpha = 0.5)
plt.scatter(X_trans[y_kmeans == 2, 0], X_trans[y_kmeans == 2, 1], s = 300, c = 'green', label = 'Cluster 3', alpha = 0.5)
plt.scatter(X_trans[y_kmeans == 3, 0], X_trans[y_kmeans == 3, 1], s = 300, c = 'cyan', label = 'Cluster 4', alpha = 0.5)
plt.scatter(X_trans[y_kmeans == 4, 0], X_trans[y_kmeans == 4, 1], s = 300, c = 'silver', label = 'Cluster 5', alpha = 0.5)
plt.scatter(X_trans[y_kmeans == 5, 0], X_trans[y_kmeans == 5, 1], s = 300, c = 'peru', label = 'Cluster 6',alpha = 0.5)
plt.scatter(X_trans[y_kmeans == 6, 0], X_trans[y_kmeans == 6, 1], s = 300, c = 'lawngreen', label = 'Cluster 7', alpha = 0.5)
plt.scatter(X_trans[y_kmeans == 7, 0], X_trans[y_kmeans == 7, 1], s = 300, c = 'lightgreen', label = 'Cluster 8', alpha = 0.5)
plt.scatter(X_trans[y_kmeans == 8, 0], X_trans[y_kmeans == 8, 1], s = 300, c = 'pink', label = 'Cluster 9', alpha = 0.5)
plt.scatter(X_trans[y_kmeans == 9, 0], X_trans[y_kmeans == 9, 1], s = 300, c = 'purple', label = 'Cluster 10', alpha = 0.5)
plt.scatter(Kmeans.cluster_centers_[:, 0], Kmeans.cluster_centers_[:, 1], s = 100, marker = '+', c = 'black', label = 'Centroids', alpha = 0.5)

for i, txt in enumerate(sample_list):
    ax.annotate(txt, (X_trans[:,0][i], X_trans[:,1][i]))
plt.legend()
#fig.savefig('C:/Users/Tamal/Documents/Thesis Files/Images/Kmeans/Kmeans_Quantile_NormalizedData_PCA_reduced.png', dpi = 200)
plt.show()

### Evaluation of the learned Kmeans model on the scaled reduced dimension dataset

In [ ]:
labels = np.zeros_like(y_kmeans)
for i in range(10):
    mask = (y_kmeans == i)
    labels[mask] = mode(y[mask])[0]

acc_score = accuracy_score(y, labels)
print('The accuracy score for optimized K-means algorithm {}.'.format(acc_score))

mat = confusion_matrix(y, labels)
sns.set(rc={'figure.figsize':(12,8)})
sns.heatmap(mat.T, square = False, annot = True, fmt = 'd', cbar = False, xticklabels = set_sample, yticklabels = set_sample)
plt.xlabel('True label')
plt.ylabel('Predicted label')
#plt.savefig('C:/Users/Tamal/Documents/Thesis Files/Images/Kmeans/ConfusionMatrix_Kmeans_Quantile_NormalizedData_PCA_reduced.png', dpi = 200)
plt.show()

### Spectral clustering performed on reduced dimension scaled data, but Spectraal clustering performs poorly on classic PCA

In [ ]:
from sklearn.cluster import SpectralClustering

SCluster = SpectralClustering(n_clusters = 10, n_init = 100, affinity = 'rbf', assign_labels = 'kmeans')
y_SC = SCluster.fit_predict(X_trans)

plt.rc('font', size = 10) #setting the front size in the plot
fig, ax = plt.subplots(figsize = (14,10)) #figure size
plt.scatter(X_trans[:, 0], X_trans[:, 1], c = y_SC, s = 300, cmap='tab10')
#plt.title('Accuracy score {} for {} PCA components.'.format(acc_score, 5))
for i, txt in enumerate(sample_list):
    ax.annotate(txt, (X_trans[:,0][i], X_trans[:,1][i]))
plt.show()

### Performance of the oaptimized Kmeans model on different number of Principal Components

In [ ]:
pca_components = [*range(num_components - 15, num_components + 1)]

fig, ax = plt.subplots(5, 3, figsize=(14, 10))
ax = np.ravel(ax)
fig.subplots_adjust(hspace = 0.01, wspace = 0.1)
for i in range(15):
    Kmeans_opt = KMeans(n_clusters = 10, init = 'k-means++', max_iter = 1000, n_init = 300)
    pca_cps = PCA(n_components = pca_components[i])
    X_pca = pca_cps.fit_transform(qt_X_train)
    Kmeans_opt.fit(X_pca)
    y_kmeans_opt = Kmeans_opt.fit_predict(X_pca)
    labels = np.zeros_like(y_kmeans_opt)
    for j in range(10):
        mask = (y_kmeans_opt == j)
        labels[mask] = mode(y[mask])[0]
    acc_score = accuracy_score(y, labels)
    ax[i].scatter(X_pca[:, 0], X_pca[:, 1], c = y_kmeans_opt, s = 5, cmap='tab10')
    ax[i].title.set_text('Accuracy score {} for {} PCA components.'.format(acc_score, pca_components[i]))
plt.tight_layout()
fig.savefig('C:/Users/Tamal/Documents/Thesis Files/Images/Kmeans/different_PCA_components_Quantile_Normalized.png', dpi = 200)